# Update Project Cards

In [1]:
import os
import sys
import yaml
import pickle

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_02')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-27 08:51:57, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-27 08:51:57, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso


## Make Crosswalk

In [6]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
cross_df = v_00_scenario.road_net.links_df[['model_link_id', 'shstReferenceId']]
cross_df.head()

,model_link_id,shstReferenceId
0,4000000,00000461094d7f302e7afdcfc7ff5ba3
1,4000001,00000b467d2e08f9abf13eeafee3ed46
2,8000000,000018062272093dbaa5d49303062454
3,2000000,000018a23d1330dcfbae79f44e9fca1a
4,2000001,0000491ae6975893d350d75ed7aa5842


## Add Model Link ID

In [79]:
filename_list = ['year_2015_i80_wb_hov_ala.yml',
                'year_2015_bus_only_ggt_on_us101.yml',
                'year_2015_attributes_exclude_trucks_sr85.yml',
                'year_2015_managed_lane_i80_ramp_to_bay_bridge.yml',
                'year_2015_attributes_exclude_trucks_i580.yml',
                'year_2015_bus_only_transbay_terminal_ramps.yml']

In [83]:
filename_list = ['year_2015_bus_only_transbay_terminal_ramps.yml']

In [84]:
for filename in filename_list:
    
    print(filename)
    
    with open(os.path.join(card_dir, filename)) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    
    x = data.get('changes')
    y = x[0].get('facility').get('link')
    df = pd.DataFrame(y[0].get('shStReferenceId'))
    df = df.rename(columns={0:'shstReferenceId'})
    
    output_df = pd.merge(df, cross_df, how='left', on='shstReferenceId')
    
    add_dict = {'model_link_id':output_df['model_link_id'].to_list()}
    
    data['changes'][0]['facility']['link'].append(add_dict)
    
    outputfile = os.path.splitext(filename)[0] + '_corrected' + os.path.splitext(filename)[1]
    
    with open(os.path.join(card_dir, outputfile), 'w') as f:
        data = yaml.dump(data, f, default_flow_style=False, sort_keys=False)
    
    
    
    
    

year_2015_bus_only_transbay_terminal_ramps.yml
